In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('vega')
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Range1d
from bokeh.io import output_notebook
from bokeh.palettes import Dark2_8
from bokeh.models import Legend, LegendItem
from bokeh.palettes import Category20
from bokeh.embed import file_html
from bokeh.resources import CDN
import copy
import requests
import json
from datetime import timedelta, date, datetime

## Import the latest COVID data, cleaned

In [14]:
df= pd.read_csv('../data/interim/cumulative_coviddata.csv', index_col=0, parse_dates=['date'])
df_daily=pd.read_csv('../data/interim/daily_coviddata.csv', index_col=0, parse_dates=['date'])
per100Th_cumulative=pd.read_csv('../data/interim/cumulative_PHT_coviddata.csv', index_col=0, parse_dates=['date'])
per100Th_daily=pd.read_csv('../data/interim/daily_PHT_coviddata.csv', index_col=0, parse_dates=['date'])

## Create widget+plot pane for visualisation of country data

In [15]:
countries = sorted(pd.unique(per100Th_daily['Country']))
country = pn.widgets.Select(name='Country', options=countries)
parameters = ['confirmed','deaths', 'recovered']
#parameters.append('all')
parameter = pn.widgets.Select(name='parameter', options=parameters)
PHT = pn.widgets.RadioButtonGroup(name='Per hundred thousand', options= ['Per hundred thousand','Total Number'], button_type ='success')
daily = pn.widgets.RadioButtonGroup(name='daily', options= ['day change','cumulative'], button_type ='success')
aggregated =pn.widgets.RadioButtonGroup(name='aggregated', options= ['daily value','7-day average'], button_type ='success')

title = '# Covid data over time'
subtitle = 'Cases or death by country'

@pn.depends(country.param.value, parameter.param.value, PHT.param.value, daily.param.value, aggregated.param.value)
def get_plot(country, parameter, PHT, daily, aggregated): 
    if daily== 'day change' and PHT == 'Per hundred thousand':
        chosen = copy.deepcopy(per100Th_daily.dropna())
    if daily == 'cumulative' and PHT == 'Per hundred thousand':
        chosen = copy.deepcopy(per100Th_cumulative)
    if daily == 'day change' and PHT == 'Total Number':
        chosen = copy.deepcopy(df_daily.dropna())
    if daily== 'cumulative' and PHT=='Total Number':
        chosen = copy.deepcopy(df)
    group2 =chosen[chosen['Country']==country][[parameter, 'date']]
    if aggregated=='7-day average':
        group2=group2.set_index('date').resample('7D').mean()
        group2.reset_index(inplace=True)
  
    else:
            group2=group2
    cds = ColumnDataSource(group2)
    hover = HoverTool(
            tooltips=[
                ("Country", "$Country"),
                ('date','@date')
        ]
    )
    p = figure(plot_width=500, plot_height=300, tools=[hover],x_axis_type='datetime')
    p.line('date', parameter, source=cds)
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = parameter
    #p.x_range = Range1d(1, 8, bounds=(1, 8))
    #p.y_range = Range1d(10, 50, bounds=(10, 50))
    p.title.text = "Country time series"
    p.title.align = "center"
    p.title.text_font_size = "12px"
    return p


newObj=pn.Column(title, 
pn.Row(pn.Column(subtitle, country, parameter, PHT, daily, aggregated), get_plot))

In [16]:

newObj

Column
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] Select(name='Country', options=['Afghanistan', ...], value='Afghanistan')
            [2] Select(name='parameter', options=['confirmed', ...], value='confirmed')
            [3] RadioButtonGroup(button_type='success', name='Per hundred thousand', options=['Per hundred thousand', ...], value='Per hundred thousand')
            [4] RadioButtonGroup(button_type='success', name='daily', options=['day change', ...], value='day change')
            [5] RadioButtonGroup(button_type='success', name='aggregated', options=['daily value', ...], value='daily value')
        [1] ParamFunction(function)